In [4]:
import re, pickle, os, string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np 
import pandas as pd 
import nltk
import string
import spacy
from nltk.corpus import stopwords
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.tokenizer import Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [5]:
import re, pickle, os, string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np 
import pandas as pd 
import nltk
import string
import spacy
from nltk.corpus import stopwords
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.tokenizer import Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [6]:
train_data = None
test_data = None

def load_data():
    global train_data, test_data
    train_data = pd.read_csv('split_data_original.txt', delimiter="\t", header=None)
    test_data = pd.read_csv('test_tweets_unlabeled.txt', delimiter="\t", header = None)
    
load_data()
train_data.columns = ['Tweet', 'label']
print(train_data)

                                                 Tweet  label
0                                                Tweet  label
1       @handle Let's try and catch up live next week!   8746
2    Going to watch Grey's on the big screen - Thur...   8746
3    @handle My pleasure Patrick....hope you are well!   8746
4    @handle Hi there! Been traveling a lot and lot...   8746
5    RT @handle Looking to Drink Clean & Go Green? ...   8746
6    RT @handle: Ft. Hood officials confirm the 2 o...   8746
7    RT @handle: Mickey Mouse is Getting a Make Ove...   8746
8             @handle How did u get the invite Justin?   8746
9    @handle I think I am still a good friend of he...   8746
10   @handle I remember! I am fine - how are u? Wha...   8746
11      @handle That's great - good for the coach!!!!!   8746
12   @handle I don't want to picture u sitting on i...   8746
13   @handle D- Thanks for the RTs....are you going...   8746
14            @handle Grrr....you must be going crazy!   8746
15   @ha

In [7]:
def load_pickle(filepath):
    documents_f = open(filepath, 'rb')
    file = pickle.load(documents_f)
    documents_f.close()
    
    return file

def save_pickle(data, filepath):
    save_documents = open(filepath, 'wb')
    pickle.dump(data, save_documents)
    save_documents.close()

In [10]:
stp = [word for word in list(stopwords.words('english') + [ "'s", "'m", "ca"])
        if word not in ["no", "not"] and word.rfind("n't") == -1]

class PreProcessor(object):
    '''Pre-processor which cleans text, lemmatises, removes stop words and punctuation, 
    returns df of processed text.'''

    def __init__(self):
        self._stopWordList = stp
        self._punct_removal = list(string.punctuation)
        self.sid = SentimentIntensityAnalyzer()


    def remove_url(self, text):
        result = re.sub(r"http\S+", "", text)
        return result
    
    def check_url(self, text):
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text) 
        if len(url) != 0:
            return True
        return False
    
    def unique_words(self, words):
        word_count = len(words)
        unique_count = len(set(words))
        if word_count!=0:
            return unique_count / word_count
        return 0
    
    def find_tags(self, text):
        from nltk.tag import StanfordNERTagger
        from nltk import word_tokenize
        
        jar = '/Users/belle/Downloads/stanford-ner/stanford-ner.jar'
        model = '/Users/belle/Downloads/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz'
        
        st = StanfordNERTagger(model, jar, encoding='utf8')

        for i in range(1):
            classified_text = st.tag(word_tokenize(text))
            print(classified_text)
        return classified_text
    
    
   
    def transform_text1(self, data):
        no_punct_translator=str.maketrans('','',string.punctuation)
        data['words'] = data['Tweet'].apply(lambda row: self.remove_url(str(row))).apply(lambda t: nltk.word_tokenize(t.translate(no_punct_translator).lower()))
        #print("data['words']=" + str(data['words']))
        #aa = [data['words']]
        #print("aa=" + str(aa))
        
        #print("data=" + str(data))
        
        
        ### tokenized
        tokenizedWord = sum(data['words'],[])
        #print("tokenizedWord=" + str(tokenizedWord))
        
        ### remove stop word
        rmStopword = [w for w in tokenizedWord if not w in stp]
        
        ### connecting all filtered words
        all_words = ' '.join([text for text in rmStopword])
        all_list= [' '.join([text for text in rmStopword])]
        #print("all_words="+all_words)
        #print("all_list="+str(all_list))
      
        ### create the word cloud
        from wordcloud import WordCloud
        wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

        plt.figure(figsize=(10, 7))
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.show()    
        
        ### count the word frequency
        a = nltk.FreqDist(rmStopword)
        d = pd.DataFrame({"Word": list(a.keys()),
                          "Count": list(a.values())})
        # selecting top 10 most frequency      
        d = d.nlargest(columns="Count", n = 20) 
        plt.figure(figsize=(16,5))
        ax = sns.barplot(data=d, x= "Word", y = "Count")
        ax.set(ylabel = "Count")
        plt.show()
             
        ### bag-of-words feature matrix
        from sklearn.feature_extraction.text import CountVectorizer
        
        #aa = [" ".join(context) for context in data['words'].values]
        aa = [" ".join(context) for context in data['words'].values]
        #print("aa=" + str(aa))
       
        bow_vectorizer = CountVectorizer(max_features=10, stop_words='english')
        bow = bow_vectorizer.fit_transform(aa) 
        bowArray = bow.toarray()
        #get_feature_names()可获取词袋中所有文本的关键字
        #toarray()可看到词频矩阵的结果
        print(bowArray)
        print (bow_vectorizer.get_feature_names())
    
 
        ### TF-IDF
        from sklearn.feature_extraction.text import TfidfVectorizer
        tfidf_vectorizer = TfidfVectorizer(max_features=10, stop_words='english')
        # TF-IDF feature matrix
        tfidf = tfidf_vectorizer.fit_transform(aa)
        

        return data

In [11]:
processor = PreProcessor()
clean_train = processor.transform_text1(train_data)
clean_train.head()

ModuleNotFoundError: No module named 'wordcloud'

In [2]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yunqiangpu/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yunqiangpu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True